# OLYMPICS ANALYSIS
ABHINAB SHARMA

__OLYMPICS__: The modern Olympic Games or Olympics are leading international sporting events featuring summer and winter sports competitions in which thousands of athletes from around the world participate in a variety of competitions. The Olympic Games are considered the world's foremost sports competition with more than 200 nations participating.The Olympic Games are held every four years, with the Summer and Winter Games alternating by occurring every four years but two years apart.
__Source: Wikipedia__

The dataset has each and every olympic event. 

I'm going to do SOME LITTLE analysis on this data completely using only pandas.

You will be seeing lot of usage of groupby method.

__NOTE:__ Ignore __Spelling__ and __grammatical__ mistakes.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import pylab as plt
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(16,12)})

In [ ]:
df = pd.read_csv("../input/athlete_events.csv")
len(df)

In [ ]:
df.head()

In [ ]:
df.info()

The dataset in pretty structured and clear.
- ID is the Unique indentifier of every participant.
- NOC is the National Olympic committe of their country.
- Team is the name of team that participated, generally it is name of country, but it might have old country names/ countries before partition etc and most importantly it ha team names of individual participants also

Lets see the Count of Team names and NOC.

In [ ]:
print("Number of Nationalities -->",len(df.NOC.unique()))
print("Number of Teams -->",len(df.Team.unique()))

I will show a glimpse of few team names.

In [ ]:
df.Team.value_counts().tail() #Look at the odd Team names

Check for Null values

In [ ]:
df.isnull().sum()

Some felt shy giving their Age, Height and Weight. Lol! Kidding. It represents data not recorded.

The Medal column has nan where the player in event has no awarded no Medal in that Event. Obviously the number will be high.

#### Lets look at the Medals distribution.

In [ ]:
df.Medal.value_counts() 

Quite obvious that Gold,Silver and Bronze are almost equal

## Total number of Unique players participated in Olympics.

In [ ]:
print("total number of Unique players participated in Olympics are -->",len(df.ID.unique()))

You might be wondering that len(df) is 271116 which is more than number of players, it's because a single player might participate in different events and also in multiple olympics.

# NOTE:
Before any further analysis, I want you to notice the columns 'Games','Year' and 'Season'.
- There are 'Winter' and 'Summer' olympics.
- Summer Olympics started in 1896 while the Winter olympics started in 1924.
- From 1924 to 1994 both were held in same year, but from 1994 the Winter olympics are held two years prior.
- Winter olympics are nothing special, they have the games which are played/competed on ICE,SNOW like ice skating, ice hockey etc.

The code below explains !

In [ ]:
print("SUMMER OLYMPICS WERE CONDUCTED IN ",np.array(sorted(df[df['Season'] == 'Summer']['Year'].unique())))

In [ ]:
print("WINTER OLYMPICS WERE CONDUCTED IN ",np.array(sorted(df[df['Season'] == 'Winter']['Year'].unique())))

## Number of Players participated in each olympics.

- Groupby year and ID, group using count()
- Taking only 'Year','ID' columns as groupby groups all the numerical columns which we dont need them here.
- as_index is false, which returns the grouping column as column, if true returns as index.

In [ ]:
groupedYearID = df.groupby(['Year','ID'],as_index=False).count()[['Year','ID']]

In [ ]:
groupedYearID = groupedYearID.groupby('Year',as_index=False).count()

In [ ]:
groupedYearID.head()

## Club the winter olympics after 1992 to its next summer olympics.
- As explained above, you need to club them to because an olympic event is considered as combined summer and winter olympics.
- The following has two for loops because you cannot update the dataframe element without using iloc.
- But, to use iloc we need index, to get the index we used the loop.
- The second loop uses those index and changes(Adds) the year column by 2.
- So, 1994 becomes 1996, 1998 becomes 2000 and so on.

In [ ]:
l = []
for i in [1994,1998,2002,2006,2010,2014]: #The year of winter olympics
    l.append(groupedYearID[groupedYearID.Year == i].index[0])

In [ ]:
for i in l:
    groupedYearID.loc[i,'Year'] = groupedYearID.loc[i,'Year'] +2

In [ ]:
groupedYearID = groupedYearID.groupby('Year',as_index=False).sum()

In [ ]:
import matplotlib.pyplot as pyplot

In [ ]:
sns.set(rc={'figure.figsize':(18,12)})
plot1 = sns.barplot('Year','ID',data=groupedYearID).set_xticklabels(groupedYearID.Year,rotation=82)
#plot1.set(xlabel='YEAR',ylabel='Number of people')
pyplot.xlabel("YEAR")
pyplot.ylabel("PARTICIPANTS")

###  Number of participants in each olympics graph. (Shown Above)

#### Now, Lets do some Gender analysis
- To get the count, we need to convert the categorical Sex column to Numerical.
- There are many ways, I chose to do using get_dummies method.
- Groupby according to ID and Sex
- It might look complicated, so I advise you to run the same code in your kernel and print it to understand.

Create the dummies and add it to the dataframe. And then group it using Year and ID using Sum.

In [ ]:
groupedGender = pd.concat([df,pd.get_dummies(df.Sex)],axis=1).groupby(['Year','ID'],as_index = False).sum()

We have grouped using sum, it will sum the M/F columns for each ID which is not correct. Look at head to understand.

In [ ]:
groupedGender[['Year','ID','F','M']].head()

In [ ]:
groupedGender.F = groupedGender.F.apply(lambda x: 0 if x==0 else 1)
groupedGender.M = groupedGender.M.apply(lambda x: 0 if x==0 else 1)

Now, look at the head of dataframe.

In [ ]:
groupedGender[['Year','ID','F','M']].head()

In [ ]:
groupedGender = groupedGender.groupby('Year',as_index=False).sum()

In [ ]:
# same code as mentioned above some where, the years of winter olympics after 1994 to be clubbed to their next summer olympics.
for i in l:
    groupedGender.loc[i,'Year'] = groupedGender.loc[i,'Year'] +2

To sum the newly edited Year columns [1994,1998,2002,2006,2010,2014]

In [ ]:
groupedGender = groupedGender.groupby('Year',as_index=False).sum()

In [ ]:
plt.plot(groupedGender.Year,groupedGender.M)
plt.plot(groupedGender.Year,groupedGender.F,color='red')

plt.plot(groupedGender.Year,groupedGender.M,'bo')
plt.plot(groupedGender.Year,groupedGender.F,'bo',color ='red')

plt.legend(['Male','Female'])
plt.xlabel("YEAR")
plt.ylabel("PARTICIPANTS")

# One good thing to notice is that the female participants are greatly increasing every year.  WOMEN EMPOWERMENT 

## Lets create separate columns for gold,silver and bronze for easy analysis.
- Using get_dummies method.
- Add newcolumn allmedals, which represents if the player in event won any medal or not.

In [ ]:
df = pd.concat([df,pd.get_dummies(df.Medal)],axis=1)

In [ ]:
df['allmedals'] = df['allmedals'] = df['Bronze'] + df['Gold'] + df['Silver'] 
#Obviously it would be either 1 or 0. Added this column to make analysis easier

In [ ]:
df.head()

## Countries with most number of medals won in all Olympics.
- Group by countries.
- Sort according to number of total medals and take top 50 of them.

In [ ]:
groupcountry = df.groupby(by=['NOC'],as_index= False).sum()

In [ ]:
top50 = groupcountry.sort_values(by=['allmedals'],ascending = False).head(50)

In [ ]:
plot2 = sns.barplot('NOC','allmedals',data=top50).set_xticklabels(top50.NOC,rotation=82)

Complete United States dominance. We all know that USA have best atheletes. 

## Countries who won most medals in each year.
- Group by Year and NOC.
- Get in index of winter olympics after 1992 and club them to their next summer olympics(as we did before)
- Create a dummy dataframe and add the country with most medals that year.

In [ ]:
groupYearNOC = df.groupby(by=['Year','NOC'],as_index=False).sum()

In [ ]:
l1 = []
for i in [1994,1998,2002,2006,2010,2014]: #The year of winter olympics
    l1.append(np.array(groupYearNOC[groupYearNOC.Year == i].index))

In [ ]:
for i in l1:
    groupYearNOC.loc[i,'Year'] = groupYearNOC.loc[i,'Year'] +2

In [ ]:
groupYearNOC = groupYearNOC.groupby(by=['Year','NOC'],as_index=False).sum()

In [ ]:
yeartop = pd.DataFrame() 
y = df.Year.unique() #Gets the Year numbers
for i in y:
    yeartop = pd.concat([yeartop,groupYearNOC[groupYearNOC['Year'] == i].sort_values(by=['allmedals'],ascending= False).head(1)])
    

- For a particular year, it sorts according to allmedals column in descending order, so the country with most medals that year will be at top. 
- So, taking head(1) means taking only the top row. 
- Then add to our new dataframe.

In [ ]:
import pylab as plt
fig, ax = plt.subplots()

sns.barplot('Year','allmedals',hue='NOC',data = yeartop,ax=ax)

# CODE COPIED FROM STACK OVERFLOW: THE CODE FOR CHANGING THE BAR'S WIDTH IN BARPLOT
def change_width(ax, new_value) :
    for patch in ax.patches :
        current_width = patch.get_width()
        diff = current_width - new_value

        # we change the bar width
        patch.set_width(new_value)

        # we recenter the bar
        patch.set_x(patch.get_x() + diff * .5)

change_width(ax, .50)
plt.show()

The years and the country which dominated that olympics.
- Note 1: I'm considering the total medals won, but in general table top will be the one with most golds.
- Note 2: The group event medals are considered as only one medal, but I'm not doing that here,im taking total number of medals won.
- Example: If a team won a medal in football, which has 11 players (5+ sub players), wher each will be awarded a medal, but as per olympics it is considered as only one medal. In our case here, I'm considering the total medals.

## The Sport and the Country that dominated that sport.
- Groupby sport and NOC.
- Create a dummy dataframe and add the country with most medals in that sport.

In [ ]:
groupsport = df.groupby(by=['Sport','NOC'],as_index=False).sum()

In [ ]:
spotop = pd.DataFrame()
sp = df.Sport.unique()
for i in sp:
    spotop = pd.concat([spotop,groupsport[groupsport['Sport'] == i].sort_values(by=['allmedals'],ascending= False).head(1)])


In [ ]:
spotop = spotop[['Sport','NOC','Bronze','Gold','Silver','allmedals']]
spotop

The name of Sport and the country which dominated that Sport.

Lets, now look at the Sports where USA shows dominance.

In [ ]:
set(spotop[spotop['NOC'] == 'USA']['Sport'])

These are sports where USA dominated in olympics. I was expecting baseball in the list, but no it is not there? :0

Lets see who tops Baseball.

In [ ]:
spotop[spotop['Sport']=='Baseball']

COOL! Didnt expect this one tho. CUBA

Lets see what all the sports that were played in Olympics so far.

In [ ]:
df.Sport.unique()

Honestly! Didnt even know half of them.

## Lets see the average Age of players in each Olympics

In [ ]:
dfage = df.groupby(['Year','ID'],as_index=False).mean()

- As, we have NaN in Age column, lets not consider them. The following fucntion does that.
- Groupby the mean 

In [ ]:
dfage = dfage[np.isfinite(dfage['Age'])]

In [ ]:
dfage = dfage.groupby('Year',as_index= False).mean()

In [ ]:
plt.plot(dfage.Year,dfage.Age)
plt.xticks(range(1896,2024,4))
plt.show()

### Plot of average age of players in each olympics

## Lets see the average Age,Height,Weight of players in their respective Sport

In [ ]:
dfagesport = df.groupby(['Sport','ID'],as_index=False).mean()

In [ ]:
dfagesport = dfagesport.groupby(['Sport'],as_index=False).mean()

In [ ]:
plot4 = sns.barplot('Sport','Age',data = dfagesport.sort_values('Age')).set_xticklabels(dfagesport.sort_values('Age').Sport,rotation=82)

#### The plot of Average of age of participants in each sport is shown above.
- To remove the Nan height columns, we do the following

In [ ]:
dfheight = dfagesport[np.isfinite(dfagesport['Height'])]

In [ ]:
plot4 = sns.barplot('Sport','Height',data = dfheight.sort_values('Height')).set_xticklabels(dfheight.sort_values('Height').Sport,rotation=82)

### The plot of Average of Height of participants in each sport is shown above.


## Obviously, Basketball players are tall.

In [ ]:
dfweight = dfagesport[np.isfinite(dfagesport['Weight'])]
plot4 = sns.barplot('Sport','Weight',data = dfweight.sort_values('Weight')).set_xticklabels(dfweight.sort_values('Weight').Sport,rotation=82)

### The participants of Tug-of-war(Now not a game of Olympics) are heavy, ummm...understandable.

#  STATS ON/OF PARTICIPANTS. 

### Top 15 Participants as per most appearances any event in Olympics

In [ ]:
df.Name.value_counts()[:15]

Shocking that he appeared more than 58 times. Lets look at his stats.

The number of times he won a medal.

In [ ]:
df[(df.Name == 'Robert Tait McKenzie') & (df.allmedals == 1)]

Ooops! Only one medal in 58 appearances? Bad luck tho

## Top 10 Players with Most number of medals.

In [ ]:
medals = df.groupby('Name',as_index=False).sum()

In [ ]:
mostmed = medals.sort_values(by=['allmedals'],ascending=False)

In [ ]:
mostmed.head(10)[['Name','Bronze','Gold','Silver','allmedals']]

#### The listed above players are top 10 (in order) players with most medals. Phelps tops the list with 28 :0 

Lets see how many players have won more than 10 medals(inclusive).

In [ ]:
print("Number of players with more than 10 Medals", len(mostmed[mostmed.allmedals >= 10]))

### Players with most number of GOLD medals.

In [ ]:
mostgold = medals.sort_values(by=['Gold'],ascending=False)
mostgold.head(10)[['Name','Gold']]

### List of players who just lost the Gold more than 3 times in their career. By which I mean, players who have won Silver in that event.


In [ ]:
mostmed[(mostmed.Silver) > (mostmed.Gold + 3)][['Name','Bronze','Gold','Silver','allmedals']]

#### Number of people who won Medal but no Gold.

In [ ]:
print("Number of people who won Medal but no Gold -->",len(mostmed[((mostmed.Silver) +( mostmed.Bronze) > 1) & (mostmed.Gold == 0)]))

#### People who won a mimum of 5 Medals but never a Gold. (Unlucky Atheletes)

In [ ]:
mostmed[((mostmed.Silver) +( mostmed.Bronze) >= 5 ) & (mostmed.Gold == 0)][['Name','Bronze','Gold','Silver','allmedals']]

#### What? Franziska van Almsick has won 10 medals but it was never a Gold.

In [ ]:
df[df.Name ==  'Franziska van Almsick'].head(1)[['Name','Sex','Age','Team','Sport']]

###### Can I call her the most unluckiest Olympic player ever? Thoughts?

### Youngest player to participate in olympics.

In [ ]:
df[df.Age ==df.Age.min()]

Gosh, he particiapted in Olympics at age of 10? LOl! Imagine what we were doing at age 10.
###### He also won a Medal in that. Which also makes him the Youngest to win a Medal in olympics.

### Oldest player to participate in olympics.

In [ ]:
df[df.Age ==df.Age.max()]

#### WHAT? really? at age 97? 
No actually, he died at age of 79(1910), his work*Art) was submitted in 1928 olympics. Not COOL, i dont count this.
#### Lets remove art competitions out of league.

In [ ]:
dfnoart = df[df.Sport !='Art Competitions']

In [ ]:
dfnoart[dfnoart.Age ==dfnoart.Age.max()]

### These two are the oldest dudes to participate ever in Olympics.
 Ohh wait! Oscar gomer won a silver. Sweet! Silver in shooting@ 72 YEARS.
#### Which also makes Oscar Gomer Swahn the Oldest Player to win a Medal.

# Players with highest success rates.
- Add new column count.
- Groupby ID or Name.
- Calculate the success rate (Medals won / count of appearances).

In [ ]:
df['countnum'] = 1
succgrp = df.groupby('Name',as_index=False).sum()

In [ ]:
succgrp['rate']  = succgrp['allmedals']/succgrp['countnum']
succgrp =succgrp.sort_values('rate',ascending = False)

### Players who participated in an event a minimum of 5 times and won medal each time they participated in an event.

In [ ]:
succgrp1 = succgrp[succgrp['countnum'] > 5]
set(succgrp1[succgrp1['rate'] == 1.0]['Name'])

### LEGENDS !!! I SAY! 100% Success rate bruh!!

### Wondering why there is no Micheal phelps name in this list? 
Well, he is legend who have 28 medals out of 30 events he participated, which means he has a success rate of 28/30 = 93%.
##### I'm only considering people with 100% hit rate.

## People who won Gold in each event they participated a minimum of 4 occasions.

In [ ]:
succgrp2 = succgrp[succgrp['countnum'] > 4]
for i in range(len(succgrp2)):
    if (succgrp2.iloc[i].Gold ==  succgrp2.iloc[i]['countnum']):
        print(succgrp2.iloc[i].Name)

## Ever heard of word 'ULTRA LEGENDS !'.

#  STATS OF __INDIA__ 
- Create a new dataframe with only INDIA records.

In [ ]:
dfindia = df[df.NOC == 'IND']

### The years india participated in Olympics


In [ ]:
sorted(dfindia.Year.unique())

# Number of Medals India Won so far.

In [ ]:
print("Total number of all Medals India won", dfindia['allmedals'].sum())

- But, Keep in mind that 197 medals are the medals every player won either solo or in group.But, the group event medals should be considered as only one medal. According to Olympics.
- Example: If a team won a medal in football, which has 11 players (5+ sub players), wher each will be awarded a medal, but as per olympics it is considered as only one medal. 
- So, lets group them by Year and Event.

In [ ]:
dfindia.groupby(['Year','Event'],as_index=False).max()['allmedals'].sum()

### Number of Olympics medals india has 29.
## BUT WAIT.
- Olympic committe says India has 28 medals, I google ans searched verywhere to find that india has 28 medals, but our analysis says we have 29.
- Well, we have won a medal in Alpinism in 1924 winter olympics as per the data I have. But it is no where mentioned in google.
- https://en.wikipedia.org/wiki/1924_Winter_Olympics Says that India was not even participated in that olympics.
## What the hell? Can someone explain why this medal was not considered.

In [ ]:
dfindia[(dfindia.Year== 1924) & (dfindia.Gold ==1)]

### Whats this mystery, i couldnt even find these players wiki pages.
# Okay! Moving forward.

#### Sports in which India won a Medal.

In [ ]:
set(dfindia[dfindia.allmedals == 1].Sport.unique())

#### Sports in which India won a Gold

In [ ]:
dfindia[dfindia.Gold == 1].Sport.unique()

- Rule out Alpinism(as we have questions about that... see above).
- Hockey is a group event, rule out
- Look at shooting.

In [ ]:
dfindia[(dfindia.Gold==1) & (dfindia.Sport == 'Shooting')]

### So, presenting you Abhinav Bindra, the person who won the first individual Gold for India.

### Lets see which years India have won the gold in hockey.

In [ ]:
dfindia[(dfindia.Gold==1) & (dfindia.Sport == 'Hockey')]['Year'].unique()

## Plotting India's performance over the years.

In [ ]:
dfindyear = dfindia.groupby(['Year','Event'],as_index=False).max()

In [ ]:
dfindyear = dfindyear.groupby(['Year'],as_index=False).sum()

In [ ]:
plt.plot(dfindyear.Year,dfindyear.allmedals)
plt.plot(dfindyear.Year,dfindyear.allmedals,'bo')
plt.yticks(range(0,8))
plt.xticks(range(1900,2018,4))
plt.show()

### Not a great looking graph. 2012 is the best year for INDIA with 6 medals.

# END NOTES:

- There's a lot more that can be extracted from this data.
- I have not higher level visualizations, they look so basic. Need to learn plotly.
- 80% of the tasks are done simply using the powerful groupby.
- Code once written was never optimized, cuz I'm dont wanna spend a lot of time in this project.
- Lastly, Please suggest me the improvizations that can be done and point out mistakes I have done.

__THANK YOU__ 

__SAITEJA NAKKA__